In [1]:
import pandas as pd
import numpy as np
import os

#讀取檔案位置與輸出檔案位置
original_path='D:/solar_eng/中興/elsie_hw/cloud_cover_original/'
processed_path='D:/solar_eng/中興/elsie_hw/cloud_cover_processed/'
#讀取資料夾中所有檔案名稱並存成list，且另存一個輸出檔案名稱list
all_original_name= os.listdir(original_path)
all_processed_name= [ os.path.splitext(i)[0]+'_processed.csv' for i in all_original_name ]

In [46]:
#設定目標檔名的關鍵字
cloudFile_name_key='cloud.fst'

print("輸出檔案：")
#使用for迴圈到all_original_name的list中抓取目前檔案名稱
for i in range(len(all_original_name)):
    #若非目標檔案則跳過不處理
    if cloudFile_name_key not in all_original_name[i]:
        continue
        
    # 讀取資料(檔名來源：all_original_name)
    df = pd.read_csv(original_path+all_original_name[i])
    #讀取經緯度
    lon=df['lon'][1]
    lat=df['lat'][1]

    
    # 將date_time欄位轉換成datetime格式
    df['fst_time'] = pd.to_datetime(df['fst_time'])


    #將重複資料刪除，只保留最後一筆資料
    duplicate_mask = df.duplicated(subset='fst_time', keep='last')
    df = df[~duplicate_mask]

    # 做填充
    df['mid'].fillna(df['hig'], inplace=True)
    df['low'].fillna(df['mid'], inplace=True)


    # 將fst_time欄位設定為索引
    df.set_index('fst_time', inplace=True)

    # 進行重取樣，將十分鐘一筆資料重取樣為5分鐘一筆
    df = df.resample('5T').first().reset_index()

    df['lon'].fillna(lon, inplace=True)
    df['lat'].fillna(lat, inplace=True)

    df[['low','mid','hig','cloud']] = df[['low','mid','hig','cloud']].interpolate()

    # 將dfst_time轉換為datetime格式
    df['fst_time'] = pd.to_datetime(df['fst_time'])

    # 選擇時間等於15、30、45或00分的資料
    df = df[df['fst_time'].dt.minute.isin([15, 30, 45, 0])]

    # 刪除date_time欄位
    df.drop('date_time', axis=1, inplace=True)

    # 將"fst_time"列名更改為"date_time"
    df = df.rename(columns={'fst_time': 'date_time'})

    # 輸出df2(檔名來源：all_processed_name)
    df.to_csv(processed_path+all_processed_name[i], index=None)
    print(all_processed_name[i])

輸出檔案：
cloud.fst.v35_solar_南鹽光_034_processed.csv
cloud.fst.v35_solar_崙尾光_033_processed.csv
cloud.fst.v35_solar_彰濱光_032_processed.csv
cloud.fst.v38_solar_南鹽光_034_processed.csv
cloud.fst.v38_solar_崙尾光_033_processed.csv
cloud.fst.v38_solar_彰濱光_032_processed.csv


In [4]:
#創建儲存power值的字典
power_dict={'南鹽光':150,'崙尾光':181,'彰濱光':100}
#設定目標檔名的關鍵字
powerFile_name_key='CF_'

print("輸出檔案：")
#使用for迴圈到all_original_name的list中抓取目前檔案名稱
for i in range(len(all_original_name)):
    #若非目標檔案則跳過不處理
    if powerFile_name_key not in all_original_name[i]:
        continue
        
    #重置power值
    power=None
    #讀取目標檔案
    df = pd.read_csv(original_path+all_original_name[i])

#若需要將capacity_factor乘上裝置容量，則將以下幾行取消註解
#     #讀取此檔案的power值，存為變數power
#     for key, value in power_dict.items():
#         if str(key) in all_original_name[i]:
#             #power為預存valu
#             power=value
#             break;
#     #確認是否有存到power值
#     if power is None:
#         print(all_original_name[i]+"缺少power值，停止轉換")
#         break;

    #讀取此檔案的經緯度
    lon=df['lon'][1]
    lat=df['lat'][1]
    

    #將格式轉為datetime並且加上10min，使其從50分成為整點資料
    df['time'] = pd.to_datetime(df['time'])+ pd.Timedelta(minutes=10)
    #將時間設為index
    df.set_index('time', inplace=True)
    #將時間間距調整為15min
    df = df.resample('15T').first().reset_index()
    #使用線性插值補上空值
#若需要將capacity_factor乘上裝置容量，將乘上power值取消註解
    df['capacity_factor']=df['capacity_factor'].interpolate()#*power
    #將經緯度的空值填滿
    df['lon'].fillna(lon, inplace=True)
    df['lat'].fillna(lat, inplace=True)

    
    # 輸出df2(檔名來源：all_processed_name)
    df.to_csv(processed_path+all_processed_name[i], index=None)
    print(all_processed_name[i])

輸出檔案：
CF_南鹽光_034_processed.csv
CF_崙尾光_033_processed.csv
CF_彰濱光_032_processed.csv


In [5]:
import pandas as pd
import numpy as np
import os

#讀取檔案位置與輸出檔案位置
#power+cloud的資料夾位置
mergeSource_path='D:/solar_eng/中興/elsie_hw/cloud_cover_processed/'
#匯出檔案的位置+檔名
mergeTarget_path='D:/solar_eng/中興/elsie_hw/solar_汙水廠_newbig_sort(history_15m).csv'
mergedOutput_path='D:/solar_eng/中興/elsie_hw/merged_processed/'

cloudFile_name_key_v35='cloud.fst.v35'
cloudFile_name_key_v38='cloud.fst.v38'
powerFile_name_key='CF_'
powerSource_name=[]
cloudSource_name_v35=[]
cloudSource_name_v38=[]
#讀取資料夾中所有檔案名稱，並將檔名分類存成power的list，和cloud的list
for i in os.listdir(mergeSource_path):
    if powerFile_name_key in i:
        powerSource_name.append(i)
    elif cloudFile_name_key_v35 in i:
        cloudSource_name_v35.append(i)
    elif cloudFile_name_key_v38 in i:
        cloudSource_name_v38.append(i)
    else:
        print('未處理檔案：'+i)
        
#讀取匯入用表格
df_target=pd.read_csv(mergeTarget_path)
#將時間欄位轉為相同格式
df_target['TIME_TO_INTERVAL']=pd.to_datetime(df_target['TIME_TO_INTERVAL'])
df_target['TIME_TO_INTERVAL'] = df_target['TIME_TO_INTERVAL'].astype(str)


for i in range(len(powerSource_name)):
    df_powerSource = pd.read_csv(mergeSource_path+powerSource_name[i])
    df_cloudSource_v35=pd.read_csv(mergeSource_path+cloudSource_name_v35[i])
    df_cloudSource_v38=pd.read_csv(mergeSource_path+cloudSource_name_v38[i])




    # 合併Power資料
    df_merged_power = pd.merge(df_target, df_powerSource, left_on='TIME_TO_INTERVAL',right_on='time', how='inner')
    # 填充空值
    df_merged_power['Power']=df_merged_power['capacity_factor']
    # 將重複欄位刪除
    df_merged_power.drop(columns=['capacity_factor','lon','lat','time'], inplace=True)

    #合併雲資料，輸出兩份檔案(v35、v38各一)
    df_merged_v35 = pd.merge(df_merged_power, df_cloudSource_v35, left_on='TIME_TO_INTERVAL',right_on='date_time', how='inner')
    df_merged_v38 = pd.merge(df_merged_power, df_cloudSource_v38, left_on='TIME_TO_INTERVAL',right_on='date_time', how='inner')
    #將v35、v38中各自的空值刪除
    df_merged_v35.drop(columns=['date_time'], inplace=True)
    df_merged_v38.drop(columns=['date_time'], inplace=True)

    sdf=df_merged_v35.iloc[29900:300101]
    pd.set_option('display.max_rows', None)
    sdf.head(100)
    #輸出df_merged_v35和df_merged_v38
    df_merged_v35.to_csv(mergedOutput_path+cloudSource_name_v35[i].replace('processed','merged'), index=None)
    df_merged_v38.to_csv(mergedOutput_path+cloudSource_name_v38[i].replace('processed','merged'), index=None)
